# ベイズ推論による機械学習

## 線型回帰

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
M = 4 # dimensions
N = 100
xmin = 0
xmax = 5
n_sample = 10
xs = np.linspace(xmin, xmax, num=N)
x_sample = xmin + (xmax-xmin) * np.random.rand(n_sample)

p_lambda = 50
epsilon = stats.norm.rvs(loc=0, scale=1.0/p_lambda, size=n_sample)

# weight
m = np.zeros(M)
l_lambda = np.identity(M)

In [ ]:
def true_func(x):
    y = 1 * np.sin(x * 0.5)
    return y
def calc_input(x):
    return [1.0, x, x**2, x**3]
def sampling_input(x, var=1.0):
    ty = true_func(x)
    return stats.norm.rvs(loc=ty, scale=np.sqrt(var))

In [ ]:
# smapling weight
l_lambda_inv = np.linalg.inv(l_lambda)

ws = stats.multivariate_normal.rvs(mean=m, cov=l_lambda_inv, size=5)

plt.figure(dpi=150)
for w in ws:
    sample_prior = [np.dot(w, calc_input(x)) for x in np.linspace(-1, 1, num=N)]
    plt.plot(np.linspace(-1, 1, num=N), sample_prior, '-')

In [ ]:
# true distribution
y_line = [true_func(x) for x in xs]

# measured data
y_true = [sampling_input(x, 1.0/p_lambda)+epsilon[i] for i, x in enumerate(x_sample)]

# plot
plt.figure(dpi=150)
plt.scatter(x_sample, y_true, label='data', color='orange')
plt.plot(xs, y_line, label='true')
plt.legend()

----
### 事後分布

$$ p(\vec{w}|\vec{Y},\vec{X})=N(\vec{w}|\vec{\hat{m}},\vec{\hat{\Lambda}}^{-1}) $$
$$ \vec{\hat{\Lambda}} = \lambda \sum^{N}_{n=1} \vec{x}_{n}\vec{x}^{T}_{n} + \vec{\Lambda} $$
$$ \vec{\hat{m}} = \vec{\hat{\Lambda}}^{-1} \lambda (\sum^{N}_{n=1} y_{n}\vec{x}_{n} + \vec{\Lambda}\vec{m}) $$

In [ ]:
print('p_lambda:', p_lambda)
l_lambda

In [ ]:
# calculation large lambda hat
sum_x = sum([np.outer(calc_input(x), calc_input(x)) for x in x_sample])
print("sum_x:", sum_x)

l_lambda_hat = p_lambda * sum_x + l_lambda
l_lambda_hat

In [ ]:
# calculation m_hat
sum_xy = sum([np.array(calc_input(x))*y for x, y in zip(x_sample, y_true)])
print('sum_xy:', sum_xy)

buff = p_lambda * sum_xy
m_hat = np.dot(np.linalg.inv(l_lambda_hat), buff)# + l_lambda * m
print("m_hat:", m_hat)

In [ ]:
# calc weight
l_lambda_hat_inv = np.linalg.inv(l_lambda_hat)
sample_post_ws = stats.multivariate_normal.rvs(mean=m_hat, cov=l_lambda_hat_inv, size=n_sample)

In [ ]:
# plot
plt.figure(dpi=150)
for w in ws:
    sample_prior = [np.dot(sample_post_ws, calc_input(x)) for x in xs]
    plt.plot(xs, sample_prior, '-')

plt.plot(xs, y_line, '--', label='true')
plt.plot(x_sample, y_true, 'o', label='sample')
plt.legend()